# Kaggle LEARN 사용자를 위한 주택 가격경쟁!
- Link [https://www.kaggle.com/c/home-data-for-ml-course/data]

In [2]:
import pandas as pd

train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')

print(len(train.columns), len(test.columns))
print(train.columns, test.columns)

81 80
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'Pave

## 데이타 컬럼
|컬럼명|의미|
|---|---|
|MSSubClass|EDA 탐색데이터|

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_data = train
test_data = test

sns.histplot(train_data["SalePrice"], kde=True, bins=30)
plt.xlabel("SalePrice")
plt.ylabel("Count")
plt.title("SalePrice Distribution")
plt.show()

sns.histplot(np.log1p(train_data["SalePrice"]), kde=True, bins=30)
plt.xlabel("Log SalePrice")
plt.ylabel("Count")
plt.title("Log SalePrice Distribution")
plt.show()

missing_values = train_data.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print(missing_values)
test_missing_values = test_data.isnull().sum()
test_missing_values = test_missing_values[test_missing_values > 0].sort_values(ascending=False)


num_features = train_data.select_dtypes(include=[np.number]).columns
train_data[num_features] = train_data[num_features].fillna(train_data[num_features].median())

test_features = test_data.select_dtypes(include=[np.number]).columns
test_data[test_features] = test_data[test_features].fillna(test_data[test_features].median())


# 범주형 변수의 결측값을 "None"으로 채움
cat_features = train_data.select_dtypes(include=[object]).columns
train_data[cat_features] = train_data[cat_features].fillna("None")

test_cat_features = test_data.select_dtypes(include=[object]).columns
test_data[test_cat_features] = test_data[test_cat_features].fillna("None")

# ID 컬럼 삭제
train_data = train_data.drop(columns=["Id"])
test_data = test_data.drop(columns=["Id"])

# 범주형 변수 One-Hot Encoding
train_data = pd.get_dummies(train_data, drop_first=True)
test_data = pd.get_dummies(test_data, drop_first=True)
print("변환된 데이터 형태:", train_data.shape)

from sklearn.model_selection import train_test_split

X = train_data.drop(columns=["SalePrice"])
common_columns = list(set(X.columns) & set(test_data.columns))
X = X[common_columns]

test_data = test_data[common_columns]

y = np.log1p(train_data["SalePrice"])  # 로그 변환 적용

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# test_data.drop(columns=["SalePrice"], inplace=True)
# 결측값 처리

# One-Hot Encoding (train과 동일한 형식으로 변환)

# train 데이터에 없던 컬럼이 test 데이터에 생길 수 있으므로 맞춰줌

# 모델 선언 및 학습
model = LinearRegression()
model.fit(X_train, y_train)


# 예측 및 평가
y_pred = model.predict(test_data)
print(y_pred)
# rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
# print("RMSE:", rmse)


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 랜덤 포레스트 모델 선언
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# 모델 학습
rf_model.fit(X_train, y_train)

# 예측 및 평가
y_pred_rf = rf_model.predict(test_data)
# rmse_rf = np.sqrt(mean_squared_error(y_valid, y_pred_rf))
# print("Random Forest RMSE:", rmse_rf)

import xgboost as xgb

# XGBoost 모델 선언
xgb_model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=42)

# 모델 학습
xgb_model.fit(X_train, y_train)

# 예측 및 평가
y_pred_xgb = xgb_model.predict(test_data)
# rmse_xgb = np.sqrt(mean_squared_error(y_valid, y_pred_xgb))
# print("XGBoost RMSE:", rmse_xgb)

import lightgbm as lgb

# LightGBM 모델 선언
lgb_model = lgb.LGBMRegressor(n_estimators=500, learning_rate=0.05, max_depth=4, random_state=42)

# 모델 학습
lgb_model.fit(X_train, y_train)

# 예측 및 평가
y_pred_lgb = lgb_model.predict(test_data)
# rmse_lgb = np.sqrt(mean_squared_error(y_valid, y_pred_lgb))
# print("LightGBM RMSE:", rmse_lgb)
print(len(y_pred_lgb))
# print(f"Linear Regression RMSE: {rmse}")
# print(f"Random Forest RMSE: {rmse_rf}")
# print(f"XGBoost RMSE: {rmse_xgb}")
# print(f"LightGBM RMSE: {rmse_lgb}")


SyntaxError: invalid syntax (1228910236.py, line 121)

In [25]:
rr = np.exp(y_pred_lgb)
ids = np.arange(1461, 1461 + len(rr))

df = pd.DataFrame({"ID": ids, "SalePrice": rr})
df.to_csv("submission.csv", index=False)

In [ ]:

np.savetxt('summit.csv', np.exp(y_pred_lgb), delimiter=',', header='ID,SalePrice')

array([143543.77884913, 332325.19273156, 116632.10136602, 153497.53551684,
       319290.97655287,  79017.57405794, 232571.78168382, 152073.92645233,
        77177.97098672, 136149.99023666, 153329.39356933, 124265.81500833,
       101642.96096755, 199588.60787225, 173394.57630462, 136456.02472053,
       191602.01214732, 136558.92128383, 113576.42590272, 210454.3579634 ,
       143309.86849617, 220173.81385452, 174440.07816767, 132816.18207558,
       200382.62275733, 160205.51979184, 197125.86519786, 107771.35477005,
       176597.62664893, 195533.20473635, 119383.90617293, 253900.62204577,
       175930.9763835 , 121241.58777608, 257139.92232763, 146591.47375534,
       118167.97445801, 210435.94849634, 333566.99835405, 106601.20856374,
       132985.01425132, 232107.6884328 , 118895.14209119, 357479.47506575,
       126717.15618091, 126906.65012103, 118836.95711985, 121407.88090439,
       394433.72731628, 137280.10375741, 122466.09578903, 213431.72626155,
       110550.86645151, 3

In [ ]:
# 데이터를 분석하자!
import matplotlib.pyplot as plt

num_plots = len(train.columns)
cols_per_row = 5
rows = -(-num_plots // cols_per_row)  # 올림 연산으로 자동 조정
fig, axes = plt.subplots(rows, cols_per_row, figsize=(20, rows * 4))

for i, col in enumerate(train.columns):
    row, col_idx = divmod(i, cols_per_row)
    ax = axes[row, col_idx] if rows > 1 else axes[col_idx]
    train[col].hist(ax=ax)
    ax.set_title(f"{col} histogram")
    ax.set_xlabel(col)
    ax.set_ylabel("Frequency")
    # train[col].hist()

    # ax.scatter(train[col], train['SalePrice'], alpha=0.5)
# plt.xlabel("Sale Price")
# plt.ylabel("Frequency")
# plt.title("Distribution of sale Price")
for j in range(i + 1, rows * cols_per_row):
    fig.delaxes(axes.flatten()[j])

plt.tight_layout()
plt.show()
# test['SalePrice'].hist()

In [ ]:
import seaborn as sns

numeric_cols = train.select_dtypes(include=['number']).columns

numeric_cols = [col for col in numeric_cols if col != 'SalePrice']

# 전체 플롯 개수
num_plots = len(numeric_cols)

# 가로 5개, 세로 자동 조정
cols_per_row = 5
rows = -(-num_plots // cols_per_row)  # 올림 연산으로 자동 계산

fig, axes = plt.subplots(rows, cols_per_row, figsize=(20, rows * 4))

# 모든 변수에 대해 scatter plot 그리기
for i, col in enumerate(numeric_cols):
    row, col_idx = divmod(i, cols_per_row)
    ax = axes[row, col_idx] if rows > 1 else axes[col_idx]  # 1행일 경우 처리
    sns.scatterplot(x=train[col], y=train["SalePrice"], ax=ax, alpha=0.5)
    ax.set_title(f"{col} vs SalePrice")

# 빈 플롯 숨기기
for j in range(i + 1, rows * cols_per_row):
    fig.delaxes(axes.flatten()[j])

plt.tight_layout()
plt.show()

In [ ]:
sns.countplot(x=train['HouseStyle'])
plt.xticks(rotation=45)  # 레이블이 많으면 회전해서 보기 좋게
plt.show()



In [ ]:
g = sns.FacetGrid(train, col="OverallQual")  # 'OverallQual' 별로 나눠서 그리기
g.map(plt.hist, "SalePrice", bins=20)  # SalePrice의 히스토그램을 각 그룹별로 그림
plt.show()


In [ ]:
sns.kdeplot(train['SalePrice'], shade=True)
plt.xlabel('Sale Price')
plt.ylabel('Density')
plt.title('KDE Plot of Sale Price')
plt.show()

In [ ]:
sns.kdeplot(train[train['OverallQual'] == 5]['SalePrice'], label="Qual 5", shade=True)
sns.kdeplot(train[train['OverallQual'] == 10]['SalePrice'], label="Qual 10", shade=True)
plt.legend()
plt.show()


In [ ]:
sns.pairplot(train[['SalePrice', 'GrLivArea', 'OverallQual']])
plt.show()


In [ ]:
sns.pairplot(train, vars=['SalePrice', 'GrLivArea', 'OverallQual'], hue="OverallQual", palette="coolwarm")
plt.show()


In [ ]:
train.info()  # 데이터 타입, Null 값 확인
train.describe()  # 기초 통계 요약
train.isnull().sum()  # 결측치 개수 확인
sns.histplot(train["SalePrice"], kde=True)
plt.show()




In [ ]:
from sklearn.model_selection import train_test_split
# print(len(test.columns), len(train.columns))
# pd.DataFrame
# print(train.columns.to_frame())

# train_features = ['YearBuilt',
# 'YearRemodAdd',
# '1stFlrSF',
# '2ndFlrSF',
# 'GrLivArea',
# 'GarageYrBlt',
# 'GarageArea',
# 'WoodDeckSF',
# 'OpenPorchSF',
# ]
# X = train[train_features]
X = train
y = train.SalePrice
# train = train.drop(columns=['SalePrice'])
train_X, val_X, train_y, val_y = train_test_split(X, y)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


price_model = RandomForestRegressor(random_state=1)
price_model.fit(train_X, train_y)
price_preds = price_model.predict(val_X)
# print(price_preds)
# price_preds = price_model.predict(t)
print(mean_absolute_error(price_preds, val_y))
# final_preds = price_model.predict(test)
#print(final_preds)
# print(test[train_features].tail())
